<a href="https://colab.research.google.com/github/SelahM87/DS4002_Group10_F2024/blob/main/SCRIPTS/Sentiment_review_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import plotly.express as px
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score
import numpy as np

In [ ]:
# Run this chunk and the next if working with raw datasets straight from the website

#Skincare_df = pd.read_csv("/Users/selahmitchell/Desktop/DS interactive app/plotly-dash/Skincare_df.csv", low_memory=False)
#productsreviews_df_1 = pd.read_csv("/Users/selahmitchell/Downloads/archive-3/reviews_0-250.csv", low_memory=False)
#productsreviews_df_2 = pd.read_csv("/Users/selahmitchell/Downloads/archive-3/reviews_500-750.csv", low_memory=False)
#productsreviews_df_3 = pd.read_csv("/Users/selahmitchell/Downloads/archive-3/reviews_750-1250.csv", low_memory=False)
#productsreviews_df_4 = pd.read_csv("/Users/selahmitchell/Downloads/archive-3/reviews_1250-end.csv", low_memory=False)
#productsreviews_df_5 = pd.read_csv("/Users/selahmitchell/Downloads/archive-3/reviews_250-500.csv", low_memory=False)

In [ ]:
#skincare_reviews_1 = productsreviews_df_1.merge(Skincare_df, left_on='product_id', right_on='product_id')
#skincare_reviews_2 = productsreviews_df_2.merge(Skincare_df, left_on='product_id', right_on='product_id')
#skincare_reviews_3 = productsreviews_df_3.merge(Skincare_df, left_on='product_id', right_on='product_id')
#skincare_reviews_4 = productsreviews_df_4.merge(Skincare_df, left_on='product_id', right_on='product_id')
#skincare_reviews_5= productsreviews_df_5.merge(Skincare_df, left_on='product_id', right_on='product_id')
#skincare_reviews_all = pd.concat([skincare_reviews_2,skincare_reviews_1,skincare_reviews_3,skincare_reviews_4,skincare_reviews_5])

In [ ]:
# Or run this chunk if you ran dataset_construction
# import dataset created from dataset_construction, replacing file path with your own

#skincare_reviews_all = pd.read_csv("/Users/selahmitchell/Desktop/DS interactive app/plotly-dash/Skincare_df.csv", low_memory=False)

In [ ]:
# filter for reviews that actually have text

#skincare_reviews_all = skincare_reviews_all[skincare_reviews_all['review_text'].notnull() & (skincare_reviews_all['review_text'] != '')]
#skincare_reviews_all.info()
#skincare_reviews_all.head()


<class 'pandas.core.frame.DataFrame'>
Index: 538514 entries, 0 to 93788
Data columns (total 23 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Unnamed: 0                538514 non-null  int64  
 1   author_id                 538514 non-null  object 
 2   rating                    538514 non-null  int64  
 3   is_recommended            442221 non-null  float64
 4   helpfulness               246073 non-null  float64
 5   total_feedback_count      538514 non-null  int64  
 6   total_neg_feedback_count  538514 non-null  int64  
 7   total_pos_feedback_count  538514 non-null  int64  
 8   submission_time           538514 non-null  object 
 9   review_text               538514 non-null  object 
 10  review_title              389069 non-null  object 
 11  skin_tone                 445340 non-null  object 
 12  eye_color                 422151 non-null  object 
 13  skin_type                 479889 non-null  object 

,Unnamed: 0,author_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,review_text,...,skin_type,hair_color,product_id,product_name,brand_name,price_usd,product_type,brand_id,product_reviews,product_rating
0,1158,9148213277,3,0.0,NaN,0,0,0,2023-03-15,A bit sticky although that’s a nice feeling (e...,...,dry,blonde,P466154,Daily Dose Vitamin C + SPF 40 Sunscreen Serum ...,Supergoop!,46.0,Sunscreen,5987,573.0,3.6736
1,1159,1306682040,4,1.0,0.5,2,1,1,2023-03-10,I really liked this formula a lot but I can’t ...,...,combination,blonde,P466154,Daily Dose Vitamin C + SPF 40 Sunscreen Serum ...,Supergoop!,46.0,Sunscreen,5987,573.0,3.6736
2,1160,5515512529,1,0.0,NaN,0,0,0,2023-03-03,"This product broke me out. Wanted to love it, ...",...,dry,black,P466154,Daily Dose Vitamin C + SPF 40 Sunscreen Serum ...,Supergoop!,46.0,Sunscreen,5987,573.0,3.6736
3,1161,5331472901,1,0.0,0.0,9,9,0,2023-02-02,when I used it 1st time - I was playing tennis...,...,normal,blonde,P466154,Daily Dose Vitamin C + SPF 40 Sunscreen Serum ...,Supergoop!,46.0,Sunscreen,5987,573.0,3.6736
4,1162,6553094889,3,0.0,0.5,2,1,1,2023-02-01,Too much of a greasy feeling id rather buy vit...,...,combination,brown,P466154,Daily Dose Vitamin C + SPF 40 Sunscreen Serum ...,Supergoop!,46.0,Sunscreen,5987,573.0,3.6736


In [ ]:
sid = SentimentIntensityAnalyzer()

# Function to calculate sentiment scores
def sentiment_scores(review):
    # Ensure it's a string
    review = str(review)
    # Split the review into sentences
    sentences = nltk.sent_tokenize(review)

    # Initialize a list to hold sentiment scores for each sentence
    scores = []

    for sentence in sentences:
        score = sid.polarity_scores(sentence)  # Get the sentiment scores
        scores.append(score['compound'])  # Append the compound score

    # Return the average compound score for the review
    return sum(scores) / len(scores) if scores else 0

# Apply the sentiment analysis to the review_text column
skincare_reviews_all['sentiment'] = skincare_reviews_all['review_text'].apply(sentiment_scores)


# Display the updated DataFrame with sentiment scores
#print(skincare_reviews_all[['review_text', 'sentiment']].head())

#skincare_reviews_all.info()
#skincare_reviews_all.head()



In [ ]:

# Calculate the average sentiment score for each product
skincare_reviews_all['average_sentiment'] = skincare_reviews_all.groupby('product_id')['sentiment'].transform('mean')
skincare_reviews_all['average_sentiment'].unique()


array([ 0.30940878,  0.37715528,  0.38199364,  0.31239265,  0.2755346 ,
        0.31671987,  0.39246923,  0.38200083,  0.24657361,  0.40854789,
        0.39656344,  0.34544217,  0.2548926 ,  0.29670658,  0.37978677,
        0.39963739,  0.28245596,  0.36193395,  0.32513726,  0.21601993,
        0.40562236,  0.35377279,  0.27791296,  0.45964015,  0.2462758 ,
        0.18122693,  0.38105937,  0.35540509,  0.34518593,  0.23548804,
        0.35897059,  0.42701666,  0.36467584,  0.27340111,  0.29285953,
        0.25465222,  0.30443303,  0.39838651,  0.34928934,  0.2882369 ,
        0.272086  ,  0.34585908,  0.15155104,  0.39201069,  0.30803799,
        0.26099389,  0.25851818,  0.24780747,  0.30822253,  0.41938629,
        0.3536218 ,  0.35290931,  0.40626737,  0.40918496,  0.34172912,
        0.32524798,  0.38185758,  0.37531337,  0.41458512,  0.45643037,
        0.16664272,  0.27380633,  0.26299949,  0.38905043,  0.33257092,
        0.26327172,  0.39363393,  0.39183854,  0.41988936,  0.27

In [ ]:
#skincare_reviews_all.to_csv('/Users/selahmitchell/Desktop/DS interactive app/plotly-dash/Skincare_df_reviews.csv', index=False)